# Model for common spatial pattern (CSP) feature calculation and classification for EEG data

> Adapted from https://github.com/MultiScale-BCI/IV-2a



In [1]:
import time

import numpy as np
from sklearn.model_selection import KFold
from sklearn.svm import LinearSVC, SVC

# import self defined functions
from csp import generate_projection, generate_eye, extract_feature
from filters import load_filterbank
from get_data import get_data


import tensorflow as tf

from tensorflow.keras import regularizers
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical


# import matplotlib.pyplot as plt
# %matplotlib inline

# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"



## Init

In [2]:
crossvalidation = False

data_path = 'data_2a_mat/'
svm_kernel = 'linear'  # 'sigmoid'#'linear' # 'sigmoid', 'rbf', 'poly'
svm_c = 0.1  # 0.05 for linear, 20 for rbf, poly: 0.1
useCSP = True
NO_splits = 5  # number of folds in cross validation
fs = 250.  # sampling frequency
NO_channels = 22  # number of EEG channels
NO_subjects = 9
NO_csp = 24  # Total number of CSP feature per band and timewindow
bw = np.array([2, 4, 8, 16, 32])  # bandwidth of filtered signals
ftype = 'butter'  # 'fir', 'butter'
forder = 2  # 4
filter_bank = load_filterbank(bw, fs, order=forder, max_freq=40,
                                   ftype=ftype)  # get filterbank coeffs
time_windows_flt = np.array([
    [2.5, 3.5],
    [3, 4],
    [3.5, 4.5],
    [4, 5],
    [4.5, 5.5],
    [5, 6],
    [2.5, 4.5],
    [3, 5],
    [3.5, 5.5],
    [4, 6],
    [2.5, 6]]) * fs  # time windows in [s] x fs for using as a feature

time_windows = time_windows_flt.astype(int)
# restrict time windows and frequency bands
# time_windows = time_windows[10] # use only largest timewindow
# filter_bank = filter_bank[18:27] # use only 4Hz bands

NO_bands = filter_bank.shape[0]
NO_time_windows = int(time_windows.size / 2)
NO_features = NO_csp * NO_bands * NO_time_windows
train_time = 0
train_trials = 0
eval_time = 0
eval_trials = 0

In [3]:
def run_csp():
    ################################ Training ############################################################################
    start_train = time.time()
    # 1. Apply CSP to bands to get spatial filter
    if useCSP:
        w = generate_projection(train_data, train_label, NO_csp, filter_bank, time_windows)
    else:
        w = generate_eye(train_data, train_label, filter_bank, time_windows)

    # 2. Extract features for training
    feature_mat = extract_feature(train_data, w, filter_bank, time_windows)

    # 3. Train SVM Model
    if svm_kernel == 'linear':
        clf = LinearSVC(C=svm_c, intercept_scaling=1, loss='hinge', max_iter=1000, multi_class='ovr',
                        penalty='l2', random_state=1, tol=0.00001)
    else:
        clf = SVC(svm_c, svm_kernel, degree=10, gamma='auto', coef0=0.0, tol=0.001, cache_size=10000,
                  max_iter=-1, decision_function_shape='ovr')
    clf.fit(feature_mat, train_label)

    end_train = time.time()

    global train_time
    global train_trials
    train_time += end_train - start_train
    train_trials += len(train_label)

    ################################# Evaluation ###################################################
    start_eval = time.time()
    eval_feature_mat = extract_feature(eval_data, w, filter_bank, time_windows)

    success_rate = clf.score(eval_feature_mat, eval_label)

    end_eval = time.time()

    print("Time for one Evaluation " + str((end_eval-start_eval)/len(eval_label)) )

    global eval_time
    global eval_trials
    eval_time += end_eval - start_eval
    eval_trials += len(eval_label)

    return success_rate

## Main

In [4]:
print("Number of used features: " + str(NO_features))

# success rate sum over all subjects
success_tot_sum = 0

print("Test data set")

start = time.time()

# Go through all subjects
for subject in range(1, NO_subjects + 1):

    print("Subject" + str(subject)+":")


    # load Eval data
    train_data, train_label = get_data(subject, True, data_path)
    eval_data, eval_label = get_data(subject, False, data_path)
    success_rate = run_csp()

    print(success_rate)
    success_tot_sum += success_rate

# Average success rate over all subjects
print("Average success rate: " + str(success_tot_sum / NO_subjects))

print("Training average time: " + str(train_time / NO_subjects))
print("Evaluation average time: " + str(eval_time / NO_subjects))

end = time.time()

print("Time elapsed [s] " + str(end - start))

Number of used features: 11352
Test data set
Subject1:


c:\users\kali\appdata\local\programs\python\python38\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


Time for one Evaluation 0.12139537156264553
0.8683274021352313
Subject2:


c:\users\kali\appdata\local\programs\python\python38\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


Time for one Evaluation 0.12081348348422101
0.5724381625441696
Subject3:


c:\users\kali\appdata\local\programs\python\python38\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


Time for one Evaluation 0.11936283722901955
0.8644688644688645
Subject4:


KeyboardInterrupt: 